In [1]:
import polynomial_multiplication as pm
import random
import math

In [12]:
n = 256
l = 56
t = 8
q = 2**23 - 2**13 + 1

coeff_limit_second_vector = (q - 1) // 16

Minimal l computation

In [14]:
# Rq = Zq[X]/(X^256 + 1)
# matrix A \in Rq^{mxk} multiplied with vector v \in Rq^k of polynomials of coefficients <= coeff_limit_second_vector

# instead of multiplying and adding up k pairs of polynomials (for each matrix row) using the kroneker+ algorithm, one may apply the first part of kroneker+ (split the ring, snort, weights, ntt, multiplication) to each pair, add up the results in the ntt domain and convert only the result back by applying the second part of kroneker+ only to the result (inverse ntt, inverse weight, sneeze, reorder coefficients). This saves k - 1 applications of the second part of the kroneker+ algorithm but the value of l needs to be slightly larger.

# another conditition is that t | l and t | n (= 256). Therefore l needs to be at least a multiple of 2. An l that is a power of 2 is very desirable because the implementation greatly simplifies.

k=5
math.log2((q-1) * coeff_limit_second_vector * 256 * k + 1)

52.31910895437802

In [4]:
def random_polynomial(coeff_limit):
    return [random.randint(0, coeff_limit - 1) for i in range(n)]

In [5]:
# random.seed(0)

a_poly = random_polynomial(q)
b_poly = random_polynomial(coeff_limit_second_vector)

In [6]:
def check_equality(p1, p2):
    for i in range(n):
        if p1[i] != p2[i]:
            print('incorrect')
            return
    
    print('correct')

In [7]:
naive_mult_res = pm.naive_multiplication(a_poly, b_poly, n, q)
print(naive_mult_res)

[4495895, 2302082, 4474174, 1475376, 7362937, 7602268, 1201720, 6036507, 654999, 7789879, 4994472, 3182563, 6247160, 6152082, 3205754, 7208444, 7275122, 3126765, 8201117, 3337235, 3105722, 5121635, 1431117, 1635798, 2690492, 6962344, 7819051, 3952562, 7066825, 7724954, 5223408, 2791119, 5300003, 4639700, 1431533, 3530441, 1835539, 3232290, 2151918, 806955, 666059, 3367192, 4422220, 8261191, 7865067, 2672370, 2016714, 6694963, 3814769, 3273140, 5898512, 5910001, 2050355, 4988306, 3038581, 3503101, 5363397, 1868987, 7041044, 4959779, 3593162, 6359092, 7430408, 5683906, 5594835, 5369734, 5822794, 3583390, 4350595, 4865297, 4898741, 5851240, 7701113, 86251, 6546212, 5928345, 1002218, 5604803, 3710598, 8006778, 3220170, 2724644, 7220151, 1997410, 7541357, 4763172, 5949622, 762862, 6155633, 2428822, 4648114, 8304198, 4707700, 2974178, 370525, 7131163, 6256577, 909411, 2444585, 3871857, 7566433, 7662213, 6582211, 8050821, 6471149, 6183537, 1383060, 7545609, 411718, 2184366, 2518149, 2116273, 

In [8]:
def run(forward_ntt, backward_ntt):
    res = pm.kroneker_plus(a_poly, b_poly, l, t, q, \
                            forward_ntt, \
                            backward_ntt)

    print(res)
    check_equality(res, naive_mult_res)

In [9]:
run(pm.ntt.naive.forward, pm.ntt.naive.backward)

a_nusskron [18365253565398124885372198849289303760971459058435277082158267461246580454039557996755577162995924729787945393849249798528203136466504492542546452060560431727386469350270168775947264585302144108309008689233518192880627324105557042834578041659771783747374998335754330653497620304519669268870448094475892940660294032026675635018635219912186208522140012251967352553232643035565658769068268189168103979503846076154317651259427370558568739066310140624700685874148833694029825243214386680929445076502787837167103315163581654618096398132, 838043781172833369678617692925742148246083358227236563541392351967141077686837446466800820068710160286804944319100518822419848660599209259796000680622609708807938625212298172700089901096489565028096152690474209261227210037496362936744000426872225179296293218954221242303194713620119908769139352524886602264969734540638681966552082578045694952167107452097448355544528361457976571446687716437698118508314770310050649999027784571185055544637645197769471386363

In [10]:
run(pm.ntt.fft_recursive.forward, pm.ntt.fft_recursive.backward)

a_nusskron [18365253565398124885372198849289303760971459058435277082158267461246580454039557996755577162995924729787945393849249798528203136466504492542546452060560431727386469350270168775947264585302144108309008689233518192880627324105557042834578041659771783747374998335754330653497620304519669268870448094475892940660294032026675635018635219912186208522140012251967352553232643035565658769068268189168103979503846076154317651259427370558568739066310140624700685874148833694029825243214386680929445076502787837167103315163581654618096398132, 838043781172833369678617692925742148246083358227236563541392351967141077686837446466800820068710160286804944319100518822419848660599209259796000680622609708807938625212298172700089901096489565028096152690474209261227210037496362936744000426872225179296293218954221242303194713620119908769139352524886602264969734540638681966552082578045694952167107452097448355544528361457976571446687716437698118508314770310050649999027784571185055544637645197769471386363

In [11]:
run(pm.ntt.fft_cyclic_ct_ct.forward, pm.ntt.fft_cyclic_ct_ct.backward)

a_nusskron [18365253565398124885372198849289303760971459058435277082158267461246580454039557996755577162995924729787945393849249798528203136466504492542546452060560431727386469350270168775947264585302144108309008689233518192880627324105557042834578041659771783747374998335754330653497620304519669268870448094475892940660294032026675635018635219912186208522140012251967352553232643035565658769068268189168103979503846076154317651259427370558568739066310140624700685874148833694029825243214386680929445076502787837167103315163581654618096398132, 838043781172833369678617692925742148246083358227236563541392351967141077686837446466800820068710160286804944319100518822419848660599209259796000680622609708807938625212298172700089901096489565028096152690474209261227210037496362936744000426872225179296293218954221242303194713620119908769139352524886602264969734540638681966552082578045694952167107452097448355544528361457976571446687716437698118508314770310050649999027784571185055544637645197769471386363